In [1]:
import tensorflow as tf
import numpy as np

# tf.enable_eager_execution()
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [14]:
impression_embed_size = 50 # 853,540개 
poi_embed_size = 50      # 13,352개
filter_embed_size = 50   # 208개
platform_embed_size = 5       # 55개
city_embed_size = 50          # 34,752개
sort_embed_size = 50 #interaction 빼고 one-hot
device_embed_size = 3 # mobile, desktop, tablet one-hot

rnn_input_size = impression_embed_size + 5 # action 5개로 통합 후 one-hot 

#LSTM
hidden_state_size = 64

In [3]:
# Parameters
learning_rate = 0.01
seq_length = 10
num_epoch = 10
batch_size = 1

input_size = 100 # TBD

In [4]:
Act_list = tf.placeholder(tf.string, [None, seq_length])
Ref_list = tf.placeholder(tf.string, [None, seq_length])

In [5]:
import pandas as pd
import functions as f
from collections import Counter

In [6]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
meta_df = pd.read_csv('./item_metadata.csv')
submit_df = pd.read_csv('./submission_popular.csv')

popular_df = f.get_popularity(train_df)

In [7]:
poi_idx = np.load('./npy/poi_names.npy', allow_pickle=True)
poi_idx = list(poi_idx)
impressions_idx = np.load('./npy/impressions_index.npy', allow_pickle=True)
impressions_idx = list(impressions_idx)
city_idx = np.load('./npy/city_names.npy', allow_pickle=True)
city_idx = list(city_idx)
platform_idx = np.load('./npy/platform_names.npy', allow_pickle=True)
platform_idx = list(platform_idx)
filter_idx = np.load('./npy/filter_merged.npy', allow_pickle=True)
filter_idx = list(filter_idx)
action_idx = np.load('./npy/action_type_names.npy', allow_pickle=True)
action_idx = list(action_idx)

sort_order_idx = np.load('./npy/sorting_names.npy', allow_pickle=True)
sort_order_idx = list(sort_order_idx)
device_idx = np.load('./npy/device_names.npy', allow_pickle=True)
device_idx = list(device_idx)

poi_embedding = tf.get_variable("poi_embedding", [len(poi_idx), poi_embed_size])
impression_embedding = tf.get_variable("impression_embedding", [len(impressions_idx), impression_embed_size])
city_embedding = tf.get_variable("city_embedding", [len(city_idx), city_embed_size])
platform_embedding = tf.get_variable("platform_embedding", [len(platform_idx), platform_embed_size])
filter_embedding = tf.get_variable("filter_embedding", [len(filter_idx), filter_embed_size])

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
train_label = train_df[(train_df["action_type"] == "clickout item")]     #train_label : (1,586,586, 12)
batch_indexes = np.random.permutation(len(train_label))

In [13]:
def embedding_function(domain, key):
    
    embedded_vector = None
    
    if domain == action_idx[9]:  #'poi'
        if key in poi_idx:
            idx = poi_idx.index(key)
        else:
            idx = 0
        embedded_vector = tf.nn.embedding_lookup(poi_embedding, idx)
        
    elif (domain == action_idx[1])|(domain == action_idx[3])|(domain == action_idx[4])|(domain == action_idx[5])\
    |(domain == action_idx[6])|(domain == action_idx[8]):  #impression
        if key in impressions_idx:
            idx = impressions_idx.index(key)
        else:
            idx = 0
        embedded_vector = tf.nn.embedding_lookup(impression_embedding, idx)
    
    elif domain == action_idx[7]: # search for destination, 'city'
        if key in city_idx:
            idx = city_idx.index(key)
        else:
            idx = 0
        embedded_vector = tf.nn.embedding_lookup(city_embedding, idx)

    elif domain == 'platform':
        if key in platform_idx:
            idx = platform_idx.index(key)
        else:
            idx = 0
        embedded_vector = tf.nn.embedding_lookup(platform_embedding, idx)
        
    elif domain ==action_idx[2]:  #'filter'
        if key in filter_idx:
            idx = filter_idx.index(key)
        else:
            idx = 0
        embedded_vector = tf.nn.embedding_lookup(filter_embedding, idx)
        
    elif domain ==action_idx[0]:  #sorting
        embedded_vector = np.zeros([sort_embed_size])
        embedded_vector[sort_order_idx.index(key)] = 1
        embedded_vector = tf.convert_to_tensor(embedded_vector, dtype=tf.float32)
        
    elif domain =='device':
        embedded_vector = np.zeros([device_embed_size])
        embedded_vector[device_idx.index(key)] = 1
        embedded_vector = tf.convert_to_tensor(embedded_vector, dtype=tf.float32)
        
    return embedded_vector

In [12]:
action_idx

['change of sort order',
 'clickout item',
 'filter selection',
 'interaction item deals',
 'interaction item image',
 'interaction item info',
 'interaction item rating',
 'search for destination',
 'search for item',
 'search for poi']

In [37]:
def action_embedding(action):
    if action == action_idx[0]:
        embed = tf.one_hot(0, 5)
    elif action == action_idx[2]:
        embed = tf.one_hot(1, 5)
    elif action == action_idx[7]:
        embed = tf.one_hot(2, 5)
    elif action == action_idx[9]:
        embed = tf.one_hot(3, 5)
    else:
        embed = tf.one_hot(4, 5)
    return embed

In [35]:
def getSequenceData(act_list, ref_list, seq_length):
    embed_total = tf.zeros([1, seq_length, rnn_input_size])  #추후 1 -> batch size 변경
    for step in range(np.shape(act_list)[1]): 
        embed_ref = embedding_function(act_list[step], ref_list[step])
        embed_action = action_embedding(act_list[step])
        embed_total[:,step,:] = tf.concat((embed_action, embed_ref), axis=0)
    return embed_total

In [10]:
def getActionList(user_label, seq_length):
    step = user_label['step']
    if step > seq_length+1:
        step = seq_length+1
    label_point = user_label.name
    history_list = train_df[label_point-step+1:label_point]
    
    act_list=[]
    ref_list=[]
    for idx in range(len(history_list)):
        act_list.append(history_list.iloc[idx]['action_type'])
        ref_list.append(history_list.iloc[idx]['reference'])
    if len(act_list) < seq_length:
        act_list = np.concatenate((np.zeros([seq_length-len(act_list)]), act_list))
        ref_list = np.concatenate((np.zeros([seq_length-len(ref_list)]), ref_list))
    
    return np.reshape(act_list, [-1, seq_length]), np.reshape(ref_list, [-1, seq_length])

In [17]:
def recurrent_neural_network_model(act_list_batch, ref_list_batch):
    
    XX = getSequenceData(Act_list, Ref_list, seq_length)

    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_state_size)    
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, XX, dtype=tf.float32)   
    
    return outputs, states

In [ ]:
# Define the neural network
def context_LSTM_net(act_list_batch, ref_list_batch):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(XX_input, n_hidden_1,activation=tf.nn.relu)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, n_hidden_2,activation=tf.nn.relu)
#     layer_2_dropout = tf.layers.dropout(layer2, training=True)
    # Output fully connected layer with a neuron for each class
    layer_3 = tf.layers.dense(layer_2, n_hidden_3, activation=tf.nn.relu)
    out_layer = tf.layers.dense(layer_3, 1, activation=tf.nn.sigmoid)
    return out_layer

In [ ]:
label_df = None
for k in range(3):
    idx = batch_indexes[k*batch_size:(k+1)*batch_size]
    label_df = train_label.iloc[idx]
    for i in range(3):  # batch size
        user_label = label_df.iloc[i]
        train_label_batch = user_label["reference"]  # 클릭한 reference number
        train_candidates = user_label['impressions']
        train_candidates = f.string_to_array(train_candidates)   
        
        candidate_prices = user_label['prices']
        candidate_prices = f.string_to_array(candidate_prices)
        candidate_prices = np.array(candidate_prices).astype(int)
        
        act_list, ref_list = getActionList(user_label, seq_length)

In [18]:
k=1
i=0

In [19]:
    idx = batch_indexes[k*batch_size:(k+1)*batch_size]
    label_df = train_label.iloc[idx]

In [20]:
        user_label = label_df.iloc[i]
        train_label_batch = user_label["reference"]  # 클릭한 reference number
        train_candidates = user_label['impressions']
        train_candidates = f.string_to_array(train_candidates)   
        
        candidate_prices = user_label['prices']
        candidate_prices = f.string_to_array(candidate_prices)
        candidate_prices = np.array(candidate_prices).astype(int)

In [23]:
user_label

user_id                                                 E8L9VGGZJJB2
session_id                                             3498d5bcec072
timestamp                                                 1541545347
step                                                              11
action_type                                            clickout item
reference                                                      85097
platform                                                          UK
city                                                 Sorrento, Italy
device                                                        tablet
current_filters    Best Value|Swimming Pool (Combined Filter)|Air...
impressions        1105350|82794|153849|45573|45453|1207105|93578...
prices             142|137|129|221|230|145|169|215|135|140|181|25...
Name: 1263363, dtype: object

In [24]:
act_list, ref_list = getActionList(user_label, seq_length)

In [34]:
range(np.shape(Act_list)[1])

range(0, 10)

In [49]:
act_list = Act_list
ref_list = Ref_list
embed_total = tf.zeros([1, seq_length, rnn_input_size])  #추후 1 -> batch size 변경



In [50]:
step =0

In [51]:
embed_ref = embedding_function(act_list[step], ref_list[step])

In [54]:
embed_action = action_embedding(act_list[step])

In [57]:
sess.run(embed_action, feed_dict={Act_list:action_list})

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles. For reference, the tensor object was Tensor("Placeholder:0", shape=(?, 10), dtype=string) which was passed to the feed with key Tensor("Placeholder:0", shape=(?, 10), dtype=string).

In [56]:
tf.concat(embed_ref, embed_action)

ValueError: Tensor conversion requested dtype int32 for Tensor with dtype float32: 'Tensor("one_hot_2:0", shape=(5,), dtype=float32)'